# キャリブレーションデモ


In [12]:
PROGRAM_NAME = "07_calibration_v2"

In [13]:
from quri_parts_oqtopus.backend import OqtopusSseBackend
from oqtopus_sse_pulse.adapter import QuriAdapter, collect_payloads_from_job, collect_session_from_job
from oqtopus_sse_pulse.plotters import plot_payload

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

DEVICE = "anemone" # デバイス名 (勉強会ではanemoneに固定)
backend = OqtopusSseBackend() # OqtopusSseBackendのインスタンスを作成

def experiment(program: str) -> list:
    """
    指定した実験プログラムを実行し, 測定結果のペイロードを返す関数.
    """

    # ジョブの実行
    job = backend.run_sse(
        file_path=f"program/{program}.py",
        device_id=f"{DEVICE}-pulse",
        name=program,
    )
    print(f"job_id = {job.job_id}")

    # ジョブの完了待ち
    job.wait_for_completion()
    print(f"job_status = {job.status}  elapsed={job.execution_time}")

    # QuriAdapterの作成
    adapter = QuriAdapter(backend)

    # 測定結果の取得
    payloads = collect_payloads_from_job(
        backend=adapter,
        job_id=job.job_id,
    )
    
    return payloads

In [14]:
payloads = experiment(PROGRAM_NAME)
print(payloads)

job_id = 0695d02b-05ad-76f7-8000-97b2df5b70ff
job_status = succeeded  elapsed=954.605
[{'calib_note': {'rabi_params': {'Q36': {'target': 'Q36', 'frequency': 0.007373905325381068, 'amplitude': 2.6163792225603415, 'phase': 0.018749945733911397, 'offset': -1.2191917006092678, 'noise': 0.04132382944226265, 'angle': -0.08011381228896575, 'distance': -1.834307312965393, 'r2': 0.9990898084666935, 'reference_phase': 2.407733065468417, 'timestamp': '2026-01-06 12:40:47'}, 'Q38': {'target': 'Q38', 'frequency': 0.010975253450807833, 'amplitude': 1.6081351593267845, 'phase': 0.011483822184290254, 'offset': 0.024245602314797946, 'noise': 0.02247176133096218, 'angle': 3.860932425656591, 'distance': 3.3025519847869873, 'r2': 0.9965416146362883, 'reference_phase': -1.9581786439938216, 'timestamp': '2026-01-06 12:41:08'}, 'Q39': {'target': 'Q39', 'frequency': 0.00967486245337665, 'amplitude': 0.5785861308469642, 'phase': 0.030840758162685965, 'offset': 1.6103528113138155, 'noise': 0.028986094519495964,

In [15]:
from pprint import pprint

pprint(payloads[0])

{'calib_note': {'hpi_params': {'Q36': {'amplitude': 0.026233442584942285,
                                       'duration': 32.0,
                                       'target': 'Q36',
                                       'tau': 12,
                                       'timestamp': '2026-01-06 12:53:41'},
                               'Q38': {'amplitude': 0.06977226355246449,
                                       'duration': 32.0,
                                       'target': 'Q38',
                                       'tau': 12,
                                       'timestamp': '2026-01-06 12:53:49'},
                               'Q39': {'amplitude': 0.01996245755190936,
                                       'duration': 32.0,
                                       'target': 'Q39',
                                       'tau': 12,
                                       'timestamp': '2026-01-06 12:53:56'},
                               'Q40': {'amplitude': 0.037031874

# calib_note, props, paramsの書き出し

以下のブロックでcalib_note, props, paramsの書き出しを行うので、必要な箇所を書き換えてpushする

In [16]:
import json
import os
import yaml

if not os.path.exists("data"):
    os.makedirs("data")

with open(os.path.join("data", "calib_note.json"), "w") as f:
    json.dump(payloads[0].get("calib_note"), f, ensure_ascii=False, indent=2)
with open(os.path.join("data", "props.yaml"), "w", encoding="utf-8") as f:
    yaml.safe_dump(payloads[0].get("props"), f)
with open(os.path.join("data", "params.yaml"), "w", encoding="utf-8") as f:
    yaml.safe_dump(payloads[0].get("params"), f)